In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

rcParams.update({'figure.autolayout': True})
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

%matplotlib inline

'''
to do: 
1.) Find a way to weight the histogram counts by the error on the
measurement.

2.) Make a GUI functionality to toggle options like the above
    a.) 2 way sliding bar to choose a date range
    b.) 2 way sliding bar to choose a lmc mod range
    c.) check boxes to choose objects -- no! Just one object at a time; makes more sense
    d.) check boxes select measurement methods

Additional:
    a.) Radio button(s) for weighted by error, by recentness, or both
    b.) Add object radio buttons and a while loop similar to the methods
    loop to grab mutliple objects from NED
    
    '''

'''
assumptions: Distances without tabulated errors are excluded! 
'''

In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons, CheckButtons
%matplotlib

#methods = ["Cepheids": "yellow", "TRGB": "orange", "RR Lyrae"]
keys = ["exclusion code", "record index", "object index", "object name","dm",
        "dmerr","distance","method","refcode","SN ID","z","h0","LMCmod","date","notes"]
obj = input("Enter the name of the object for which you want to retrieve "
              "distances: ")

#colors1 = ["red", "blue", "yellow"]
#colors2 = ["green", "yellow", "cyan"]
#alphas = [1.0, 0.60, 0.80]



#########################################################################
######################## DATABASE SEARCH FUNCTIONS ######################
#########################################################################


def obj_filter(r, o):
    n = r['object name']
    if n == o or n == o.replace(" ","") or n == o.lower() or n == o.upper():
        return r 

def get_dm(r): # need separate get_dm function bc of the replacement 
    d = r['LMCmod']
    if d == "":
        return 18.5
    else: return d
    
def get_val(r, key):
    '''
    Check if user looking for the LMCmod value
    then branch from there
    
    '''
    val = r[key]
    if key == 'LMCmod':
        if val == "":
            return 18.5
        else:
            return val
    else:
        return val
    
def check_for(d, key, val):
    '''
    Simply returns True if the given value exists 
    under a certain key in the given dict. Do this
    instead of plain existence because each key 
    might not have mutually exclusive values!
    '''
    if d[key] == val or d[key] == val.replace(" ","") or d[key] == val.lower() or d[key] == val.upper():
        return True
def slice_intersect(sli1, sli2, sli3):
    sli = []
    for i, j, k in zip(sli1, sli2, sli3):
        if i and j and k:
            sli.append(True)
        else:
            sli.append(False)
    return sli

def squeeze(nums, mini, maxi):
    nums = [ v for v in nums if v > mini and v < maxi ]  
    return vals

def plot_hist(data, errs, l):
#    rel_err = errs/data
    y, binedges = np.histogram(data, bins = 15) # want to weight counts by error
    bincenters  = 0.5 * (binedges[1:] + binedges[:-1])
    width       = np.ptp(data)/15
    p = plt.bar(bincenters, y, width = width, label = l)
    return p

def hist(d, y, l, y_min, y_max, l_min, l_max, m_ind, b = 15):
    '''
    d: array of distance moduli
    
    y: array of years
    
    m_ind: method indices for the first 3 arrays
    
    b: number of bins for the histogram
    
    Returns:
    
    h: histogram values
    
    e: bin edges'''
    y_sli = ((y >= y_min) & (y <= y_max))
    y_ind = np.arange(0, len(d), step = 1)[y_sli]
    l_sli = ((l >= l_min) & (l <= l_max))
    l_ind = np.arange(0, len(d), step = 1)[l_sli]
    ind = np.array(list(set.intersection(set(y_ind), set(l_ind), set(m_ind))))
    h, e = np.histogram(d[ind], bins = b)
    return h, e        

#########################################################################
######################### Filtering NED database ########################
#########################################################################


# pull rows corresponding to specified object. Exclude dm's w/ no error #

objRows = []
with open("NED26.05.1-D-12.1.0-20160501.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        drow = dict(zip(keys, row))
        r = obj_filter(drow, obj)
        if r != None and r['dmerr']: # python empty strings are 'falsy'
            objRows.append(r)

if any(x == None for x in objRows):
    print("The object name {0} is either not present or the naming"
      " format is not correct. Try looking at a few rows"
      " in the database to find the right convention.".format(obj))
else:
    print("The query returned {0} entries for the specified object.".format(len(objRows)))

    

# get list of indices corresponding to each desired measurement method  #

methods = [] # the overall methods list to get a list of methods to plot
no = ['n', 'N', 'no', 'NO', 'No']
dms = []
dmerrs = []
ceph_ind = []
rrly_ind = []
trgb_ind = []
# need to come up with alternative to individuall naming each method's list
while True:
    t = False
    method = []
    m = input("Enter a method, like 'Cepheids' or 'TRGB', you "
            "would like to search for in the new shortened  "
            "object database. If you do not wish to search "
            "for more methods hit enter or type no: ")
    for r in objRows:
        if check_for(r, 'method', m):
            method.append(m)
            t = True # t is an "any" true to avoid duplicate methods in the final list
        if r['method'] == 'Cepheids' == m:
            ceph_ind.append(objRows.index(r))
        elif r['method'] == 'RR Lyrae' == m:
            rrly_ind.append(objRows.index(r))
        elif r['method'] == 'TRGB' == m:
            trgb_ind.append(objRows.index(r))
    if t:
        methods.append(m)
        print("{0} of the {1} total {2} results used the {3} method.".format(len(method),
                                                                        len(objRows), obj, m))   
    elif m in no or m == '':
        print("broken")
        break
    else:
        print("The method {0} is either not present or the input"
              "format is not correct. Try looking at a few rows"
              "in the database to find the right convention.".format(m))
        

# getting the needed data # 
dms = []
dmerrs = []
years = []
lmcs = []
for oRow in objRows:
    dms.append(float(get_val(oRow, 'dm')))
    dmerrs.append(float(get_val(oRow, 'dmerr')))
    years.append(float(get_val(oRow, 'date')))
    lmcs.append(float(get_val(oRow, 'LMCmod')))

dms = np.array(dms)
dmerrs = np.array(dmerrs)
years = np.array(years) + 1980
lmcs = np.array(lmcs)



#########################################################################
########################### SLIDERS AND AXES ############################
#########################################################################

def hist(d, y, l, y_min, y_max, l_min, l_max, m_ind, b = 15):
    '''
    d: array of distance moduli
    
    y: array of years
    
    m_ind: method indices for the first 3 arrays
    
    b: number of bins for the histogram
    
    Returns:
    
    h: histogram values
    
    c: bin centers
    
    wid: width of bars
    '''
    y_sli = ((y >= y_min) & (y <= y_max))
    y_ind = np.arange(0, len(d), step = 1)[y_sli]
    l_sli = ((l >= l_min) & (l <= l_max))
    l_ind = np.arange(0, len(d), step = 1)[l_sli]
    ind = np.array(list(set.intersection(set(y_ind), set(l_ind), set(m_ind))))
    h, binedges = np.histogram(d[ind], bins = b)
    c  = 0.5 * (binedges[1:] + binedges[:-1])
    wid = np.ptp(d[ind])/15
    return h, c, wid

fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.35)


# THESE are my "functions" to alter with sliders! then just plot them!! #

# want to weight counts by error

min_yr0 = 1985
max_yr0 = 2015
min_lmc0 = 18.40
max_lmc0 = 18.55

h1, bincenters1, w1 = hist(dms, years, lmcs, min_yr0, max_yr0, min_lmc0, max_lmc0, ceph_ind)
h2, bincenters2, w2 = hist(dms, years, lmcs, min_yr0, max_yr0, min_lmc0, max_lmc0, rrly_ind)
h3, bincenters3, w3 = hist(dms, years, lmcs, min_yr0, max_yr0, min_lmc0, max_lmc0, trgb_ind)

rects1 = plt.bar(bincenters1, h1, width = w1, label = 'Cepheids')
rects2 = plt.bar(bincenters2, h2, width = w2, label = 'RR Lyrae')
rects3 = plt.bar(bincenters3, h3, width = w3, label = 'TRGB')
plt.legend()
#plt.axis([0, 1, -10, 10])

axcolor = 'lightgoldenrodyellow'

# [starting point from left to right, starting point bottom to top, ending point left to right, thickness]
axmin_yr = plt.axes([0.25, 0.15, 0.65, 0.03], axisbg=axcolor)
axmax_yr = plt.axes([0.25, 0.10, 0.65, 0.03], axisbg=axcolor)
axmin_lmc = plt.axes([0.25, 0.25, 0.65, 0.03], axisbg=axcolor)
axmax_lmc = plt.axes([0.25, 0.20, 0.65, 0.03], axisbg=axcolor)

smin_yr = Slider(axmin_yr, 'Minimum Year', 1975, max(years) , valinit=min_yr0)
smax_yr = Slider(axmax_yr, 'Maximum Year', 1975, max(years), valinit=max_yr0)
smin_lmc = Slider(axmin_lmc, "Minimum LMC m - M", 18.4, max(lmcs), valinit=min_lmc0)
smax_lmc = Slider(axmax_lmc, 'Maximum LMC m - M', 18.4, max(lmcs), valinit=max_lmc0)

# use numpy histogram option "weights" to deweight more erroneous and older values!
# just use TWO sliders to squeeze values!
def update(val):
    min_yr = smin_yr.val
    max_yr = smax_yr.val
    min_lmc = smin_lmc.val
    max_lmc = smax_lmc.val
    heights1, bincenters1, width1 = hist(dms, years, lmcs, min_yr, max_yr, min_lmc, max_lmc, ceph_ind)
    heights2, bincenters2, width2 = hist(dms, years, lmcs, min_yr, max_yr, min_lmc, max_lmc, rrly_ind)
    heights3, bincenters3, width3 = hist(dms, years, lmcs, min_yr, max_yr, min_lmc, max_lmc, trgb_ind)
    for r1, r2, r3, h1, h2, h3 in zip(rects1, rects2, rects3, heights1, heights2, heights3):
        r1.set_height(h1)
        r2.set_height(h2)
        r3.set_height(h3)
    fig.canvas.draw_idle()
smin_yr.on_changed(update)
smax_yr.on_changed(update)
smin_lmc.on_changed(update)
smax_lmc.on_changed(update)
                     
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    smin_yr.reset()
    smax_yr.reset()
    smin_lmc.reset()
    smax_lmc.reset()
button.on_clicked(reset)

#rax = plt.axes([0.025, 0.5, 0.15, 0.15], axisbg=axcolor)
#radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)


#def colorfunc(label):
#    l.set_color(label)
#    fig.canvas.draw_idle()
#radio.on_clicked(colorfunc)

cax = plt.axes([0.05, 0.4, 0.125, 0.15])
check = CheckButtons(cax, ['Cepheids', 'RR Lyrae', 'TRGB'], (True, True, True))

def func(label): # remember to add or remove methods here
    if label == 'Cepheids':
        p1.set_visible(not ceph.get_visible())
    elif label == 'RR Lyrae':
        p2.set_visible(not rrly.get_visible())
    elif label == 'TRGB':
        p3.set_visible(not trgb.get_visible())
    plt.draw()
check.on_clicked(func)

plt.show()


Using matplotlib backend: MacOSX
Enter the name of the object for which you want to retrieve distances: MESSIER 031
The query returned 331 entries for the specified object.
Enter a method, like 'Cepheids' or 'TRGB', you would like to search for in the new shortened  object database. If you do not wish to search for more methods hit enter or type no: Cepheids
84 of the 331 total MESSIER 031 results used the Cepheids method.
Enter a method, like 'Cepheids' or 'TRGB', you would like to search for in the new shortened  object database. If you do not wish to search for more methods hit enter or type no: TRGB
49 of the 331 total MESSIER 031 results used the TRGB method.
Enter a method, like 'Cepheids' or 'TRGB', you would like to search for in the new shortened  object database. If you do not wish to search for more methods hit enter or type no: RR Lyrae
22 of the 331 total MESSIER 031 results used the RR Lyrae method.
Enter a method, like 'Cepheids' or 'TRGB', you would like to search for i

In [65]:
#########################################################################
########################### SLIDERS AND AXES ############################
#########################################################################

def hist(d, y, l, y_min, y_max, l_min, l_max, m_ind, b = 15):
    '''
    d: array of distance moduli
    
    y: array of years
    
    m_ind: method indices for the first 3 arrays
    
    b: number of bins for the histogram
    
    Returns:
    
    h: histogram values
    
    c: bin centers
    
    wid: width of bars
    '''
    y_sli = ((y >= y_min) & (y <= y_max))
    y_ind = np.arange(0, len(d), step = 1)[y_sli]
    l_sli = ((l >= l_min) & (l <= l_max))
    l_ind = np.arange(0, len(d), step = 1)[l_sli]
    ind = np.array(list(set.intersection(set(y_ind), set(l_ind), set(m_ind))))
    h, binedges = np.histogram(d[ind], bins = b)
    c  = 0.5 * (binedges[1:] + binedges[:-1])
    wid = np.ptp(d[ind])/15
    return h, c, wid

fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.35)


# THESE are my "functions" to alter with sliders! then just plot them!! #

# want to weight counts by error

min_yr0 = 1985
max_yr0 = 2015
min_lmc0 = 18.40
max_lmc0 = 18.55

h1, bincenters1, w1 = hist(dms, years, lmcs, min_yr0, max_yr0, min_lmc0, max_lmc0, ceph_ind)
h2, bincenters2, w2 = hist(dms, years, lmcs, min_yr0, max_yr0, min_lmc0, max_lmc0, rrly_ind)
h3, bincenters3, w3 = hist(dms, years, lmcs, min_yr0, max_yr0, min_lmc0, max_lmc0, trgb_ind)

rects1 = plt.bar(bincenters1, h1, width = w1, label = 'Cepheids')
rects2 = plt.bar(bincenters2, h2, width = w2, label = 'RR Lyrae')
rects3 = plt.bar(bincenters3, h3, width = w3, label = 'TRGB')
plt.legend()
#plt.axis([0, 1, -10, 10])

axcolor = 'lightgoldenrodyellow'

# [starting point from left to right, starting point bottom to top, ending point left to right, thickness]
axmin_yr = plt.axes([0.25, 0.15, 0.65, 0.03], axisbg=axcolor)
axmax_yr = plt.axes([0.25, 0.10, 0.65, 0.03], axisbg=axcolor)
axmin_lmc = plt.axes([0.25, 0.25, 0.65, 0.03], axisbg=axcolor)
axmax_lmc = plt.axes([0.25, 0.20, 0.65, 0.03], axisbg=axcolor)

smin_yr = Slider(axmin_yr, 'Minimum Year', 1975, max(years) , valinit=min_yr0)
smax_yr = Slider(axmax_yr, 'Maximum Year', 1975, max(years), valinit=max_yr0)
smin_lmc = Slider(axmin_lmc, "Minimum LMC m - M", 18.4, max(lmcs), valinit=min_lmc0)
smax_lmc = Slider(axmax_lmc, 'Maximum LMC m - M', 18.4, max(lmcs), valinit=max_lmc0)

# use numpy histogram option "weights" to deweight more erroneous and older values!
# just use TWO sliders to squeeze values!
def update(val):
    min_yr = smin_yr.val
    max_yr = smax_yr.val
    min_lmc = smin_lmc.val
    max_lmc = smax_lmc.val
    heights1, bincenters1, width1 = hist(dms, years, lmcs, min_yr, max_yr, min_lmc, max_lmc, ceph_ind)
    heights2, bincenters2, width2 = hist(dms, years, lmcs, min_yr, max_yr, min_lmc, max_lmc, rrly_ind)
    heights3, bincenters3, width3 = hist(dms, years, lmcs, min_yr, max_yr, min_lmc, max_lmc, trgb_ind)
    for r1, r2, r3, h1, h2, h3 in zip(rects1, rects2, rects3, heights1, heights2, heights3):
        r1.set_height(h1)
        r2.set_height(h2)
        r3.set_height(h3)
    fig.canvas.draw_idle()
smin_yr.on_changed(update)
smax_yr.on_changed(update)
smin_lmc.on_changed(update)
smax_lmc.on_changed(update)
                     
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    smin_yr.reset()
    smax_yr.reset()
    smin_lmc.reset()
    smax_lmc.reset()
button.on_clicked(reset)

#rax = plt.axes([0.025, 0.5, 0.15, 0.15], axisbg=axcolor)
#radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)


#def colorfunc(label):
#    l.set_color(label)
#    fig.canvas.draw_idle()
#radio.on_clicked(colorfunc)

cax = plt.axes([0.05, 0.4, 0.125, 0.15])
check = CheckButtons(cax, ['Cepheids', 'RR Lyrae', 'TRGB'], (True, True, True))

def func(label): # remember to add or remove methods here
    if label == 'Cepheids':
        p1.set_visible(not ceph.get_visible())
    elif label == 'RR Lyrae':
        p2.set_visible(not rrly.get_visible())
    elif label == 'TRGB':
        p3.set_visible(not trgb.get_visible())
    plt.draw()
check.on_clicked(func)

plt.show()


In [54]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
import scipy.signal as signal
%matplotlib

fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.25)
#t = np.arange(0.0, 1.0, 0.001)
#a0 = 5
#f0 = 3
#s = a0*np.sin(2*np.pi*f0*t)
#l, = plt.plot(t, s, lw=2, color='red')
#plt.axis([-30, 30, -10, 10])

n = np.random.normal(0, 10, 1000)
w0 = np.ptp(n)/15
testy, testbinedges = np.histogram(n, bins = 15)
testbincenters  = 0.5 * (testbinedges[1:] + testbinedges[:-1])
rects = plt.bar(testbincenters, testy, width = w0)

axcolor = 'lightgoldenrodyellow'
#axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], axisbg=axcolor)
#axamp = plt.axes([0.25, 0.15, 0.65, 0.03], axisbg=axcolor)
axsig = plt.axes([0.25, 0.15, 0.65, 0.03], axisbg=axcolor)

#sfreq = Slider(axfreq, 'Freq', 0.1, 30.0, valinit=f0)
#samp = Slider(axamp, 'Amp', 0.1, 10.0, valinit=a0)
ssig = Slider(axsig, 'Sigma', 5.0, 15.0, valinit=10)

def update(val):
#    amp = samp.val
#    freq = sfreq.val
    sig = ssig.val
    n = np.random.normal(0, sig, 1000)
    testy, testbinedges = np.histogram(n, bins = 15)
    testbincenters  = 0.5 * (testbinedges[1:] + testbinedges[:-1])
    w = np.ptp(n)
    for rect, y in zip(rects,testy):
        rect.set_height(y)
    fig.canvas.draw_idle()
#sfreq.on_changed(update)
#samp.on_changed(update)
ssig.on_changed(update)


resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    sfreq.reset()
    samp.reset()
button.on_clicked(reset)

rax = plt.axes([0.025, 0.5, 0.15, 0.15], axisbg=axcolor)
radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)


def colorfunc(label):
    l.set_color(label)
    fig.canvas.draw_idle()
radio.on_clicked(colorfunc)

plt.show()


Using matplotlib backend: MacOSX


In [14]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import CheckButtons
%matplotlib

t = np.arange(0.0, 2.0, 0.01)
s0 = np.sin(2*np.pi*t)
s1 = np.sin(4*np.pi*t)
s2 = np.sin(6*np.pi*t)

fig, ax = plt.subplots()
l0, = ax.plot(t, s0, visible=False, lw=2)
l1, = ax.plot(t, s1, lw=2)
l2, = ax.plot(t, s2, lw=2)
plt.subplots_adjust(left=0.2)

rax = plt.axes([0.05, 0.4, 0.1, 0.15])
check = CheckButtons(rax, ('2 Hz', '4 Hz', '6 Hz'), (False, True, True))


def func(label):
    if label == '2 Hz':
        l0.set_visible(not l0.get_visible())
    elif label == '4 Hz':
        l1.set_visible(not l1.get_visible())
    elif label == '6 Hz':
        l2.set_visible(not l2.get_visible())
    plt.draw()
check.on_clicked(func)

plt.show()

Using matplotlib backend: MacOSX


In [18]:
##### single histogram test ######

fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.35)


# THESE are my "functions" to alter with sliders! then just plot them!! #

# want to weight counts by error

min_yr0 = 1995
max_yr0 = 2015
min_lmc0 = 18.45
max_lmc0 = 18.51

h1, bincenters1, w1 = hist(dms, years, lmcs, min_yr0, max_yr0, min_lmc0, max_lmc0, ceph_ind)
p1 = ax.bar(bincenters1, h1, width = w1, label = 'Cepheids')
plt.legend()
#plt.axis([0, 1, -10, 10])

axcolor = 'lightgoldenrodyellow'

# [starting point from left to right, starting point bottom to top, ending point left to right, thickness]
axmin_yr = fig.add_axes([0.25, 0.15, 0.65, 0.03], axisbg=axcolor)
axmax_yr = fig.add_axes([0.25, 0.10, 0.65, 0.03], axisbg=axcolor)
axmin_lmc = fig.add_axes([0.25, 0.25, 0.65, 0.03], axisbg=axcolor)
axmax_lmc = fig.add_axes([0.25, 0.20, 0.65, 0.03], axisbg=axcolor)

smin_yr = Slider(axmin_yr, 'Minimum Year', 1975, max(years) , valinit=min_yr0)
smax_yr = Slider(axmax_yr, 'Maximum Year', 1975, max(years), valinit=max_yr0)
smin_lmc = Slider(axmin_lmc, "Minimum LMC m - M", 18.4, max(lmcs), valinit=min_lmc0)
smax_lmc = Slider(axmax_lmc, 'Maximum LMC m - M', 18.4, max(lmcs), valinit=max_lmc0)

# use numpy histogram option "weights" to deweight more erroneous and older values!
# just use TWO sliders to squeeze values!


def update(val):
    min_yr = smin_yr.val
    max_yr = smax_yr.val
    min_lmc = smin_lmc.val
    max_lmc = smax_lmc.val
    print(h1)
    print(bincenters1)
#    ax.cla()
    h1, bincenters1 = hist(dms, years, lmcs, min_yr, max_yr, min_lmc, max_lmc, ceph_ind)
    p1.set_data(bincenters, h1)
#    p1.set_xdata(bincenters1)
#    p1.set_ydata(h1) # this is the line that will replot. l is the artist object from plt.plot    
#    p1.draw()
    fig.canvas.draw_idle()

smin_yr.on_changed(update)
smax_yr.on_changed(update)
smin_lmc.on_changed(update)
smax_lmc.on_changed(update)

'''
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    smin_yr.reset()
    smax_yr.reset()
    smin_lmc.reset()
    smax_lmc.reset()
button.on_clicked(reset)

#rax = plt.axes([0.025, 0.5, 0.15, 0.15], axisbg=axcolor)
#radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)


#def colorfunc(label):
#    l.set_color(label)
#    fig.canvas.draw_idle()
#radio.on_clicked(colorfunc)

cax = plt.axes([0.05, 0.4, 0.125, 0.15])
check = CheckButtons(cax, ['Cepheids', 'RR Lyrae', 'TRGB'], (True, True, True))

def func(label): # remember to add or remove methods here
    if label == 'Cepheids':
        p1.set_visible(not ceph.get_visible())
    plt.draw()
check.on_clicked(func)
'''
plt.show()


In [4]:
# h1, bincenters1 = hist(dms, years, lmcs, min_yr, max_yr, min_lmc, max_lmc, ceph_ind)
print(h1)
print(h2)
print(h3)
plt.bar(bincenters1, h1, width = w1, label = 'Cepheids')

NameError: name 'h1' is not defined

In [ ]:
ss = amp*np.sin(2*np.pi*freq*t)
l.set_ydata(ss)

h1, bincenters1 = hist(dms, years, lmcs, min_yr, max_yr, min_lmc, max_lmc, ceph_ind)
p1.set_ydata(h1) # this is the line that will replot. l is the artist object from plt.plot

In [39]:
n = np.random.normal(0, 10, 1000)
#plt.plot(n)

testy, testbinedges = np.histogram(n, bins = 15)
testbincenters  = 0.5 * (testbinedges[1:] + testbinedges[:-1])
print(len(testy))
print(len(testbincenters))
plt.bar(testbincenters, testy, width = np.ptp(n)/15)

15
15


<Container object of 15 artists>